## **TIL**
### **FLASK 사용한 크롤링**
**여러가지 url 주소와 html rendering하기**
* index.html과 index_table 만들기
* `<table>`과 `<thead>` `<tbody>` `<tr>` 구조 이해
* `<button>`과 `<a href>` 활용하여 두 페이지 간 연결 

**html문서 dictionary로 변경하고 rendering**


### **데이터 베이스**
* **시스템**
* 시스템 데이터셋의 구조
    * 예) 고객 데이터 베이스(DB)
    * 고객 신상 정보 DB: 성별, 나이, 취미, 결혼 유무
    * 구매 DB, 상품 DB, 구입 거래처 DB
* **RDMBS**
* https://blog.siner.io/2021/10/11/rdbms-comparison/
* 주요 RDBMS의 종류:<table><td> mysql</td> <td> mariadb<td>  <td>postgresql</td> <td>sqlite</td> <td> oracle </td> <td>mssql<td> </table>

In [ ]:
import pandas as pd

url = "http://companyinfo.stock.naver.com/v1/company/c1010001.aspx?cmp_cd=005930"
datas = pd.read_html(url, encoding="utf-8")[12]
datas = datas.to_dict(orient="records")
datas[1:]

In [50]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm

url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D&pageNo=1&numOfRows=10&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt=20230101&endDt=20231231&stnIds=108'
xml=requests.get(url)
soup=bs(xml.text,'html.parser')
totCnt=soup.totalcount.text
totCnt

c:\Users\bluecom001\miniconda3\envs\mp\lib\site-packages\bs4\builder\__init__.py:546: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  XMLParsedAsHTMLWarning.MESSAGE, XMLParsedAsHTMLWarning


'365'

In [51]:
# 공공API 인증키 (부성순강사 인증키임. 본인자료는 다시 인증키 받아야함.)
def myUrlOpen(지점코드=108,totCnt=1,startDate='20230101',endDate='20231231'):
    인증키='Y9vZZYdTITnGhBz7r3628UaVagdVfFEt7urjSsx3ha5rmD9ohXbuD546nfVlktbIbE9G5At1lXbcXKSo%2BEYYig%3D%3D'
    지점코드=108   #서울
    url='http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList?serviceKey=' 
    url=url+인증키
    url=url+ '&pageNo=1&numOfRows='
    url=url+str(totCnt)+'&dataType=XML&dataCd=ASOS&dateCd=DAY&startDt='
    url=url+startDate+'&endDt='+endDate+'&stnIds='
    url=url+str(지점코드)
    #print(url)
    return url

In [52]:
자료=[]
지점리스트={'지점명':['서울','부산','제주'],
              '지점코드': [108,253,184]}
jijumList,dataList,minList,maxList,rnList=[],[],[],[],[]

for 지점명,지점코드 in zip(지점리스트['지점명'],지점리스트['지점코드']):

    url=myUrlOpen(지점코드,totCnt)
    xml=requests.get(url)
    soup=bs(xml.text,'html.parser')
    data=soup.find_all('item')


    for item in data:
        jijum=지점명
        date=item.find('tm').text
        min=item.find('minta').text
        max=item.find('maxta').text
        rn=item.find('sumrn').text
    
        jijumList.append(jijum) ;dataList.append(date) ; maxList.append(max); minList.append(min) ; rnList.append(rn)
    
dataList=dict({'j':jijumList,
          'd':dataList,
          'm':maxList,
          'n':minList,
          'r':rnList})

In [56]:
df=pd.DataFrame(dataList)
fileName='_'.join(지점리스트['지점명'])   # 파일명으로 사용하기 위하여
df.to_csv('c:/data/weather/날씨_'+fileName+'.csv', encoding='utf-8',index=False)
df.tail()

,j,d,m,n,r
1090,제주,2023-12-27,6.8,-2.8,
1091,제주,2023-12-28,4.4,-1.7,
1092,제주,2023-12-29,4.6,-1.5,
1093,제주,2023-12-30,2.1,-0.9,13.1
1094,제주,2023-12-31,4.3,0.6,4.7


In [57]:
import json
with open('c:/data/weather/날씨_'+fileName+'.json', 'w', encoding='utf-8') as f : 
	json.dump(dataList, f, indent='\t',ensure_ascii=False)

In [75]:
file_path = r"C:\data\weather\날씨_서울_부산_제주.json"

with open(file_path, encoding='utf-8') as file:
    data = json.load(file)

In [134]:
import pandas as pd
df = pd.read_csv('c:/data/DB_dataList/seoul.csv', encoding="euc-kr")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30948 entries, 0 to 30947
Data columns (total 17 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   기준_년분기_코드     30948 non-null  int64  
 1   상권_구분_코드      30948 non-null  object 
 2   상권_구분_코드_명    30948 non-null  object 
 3   상권_코드         30948 non-null  int64  
 4   상권_코드_명       30948 non-null  object 
 5   월_평균_소득_금액    29196 non-null  float64
 6   소득_구간_코드      29196 non-null  float64
 7   지출_총금액        30324 non-null  float64
 8   식료품_지출_총금액    30324 non-null  float64
 9   의류_신발_지출_총금액  30324 non-null  float64
 10  생활용품_지출_총금액   30324 non-null  float64
 11  의료비_지출_총금액    30324 non-null  float64
 12  교통_지출_총금액     30324 non-null  float64
 13  여가_지출_총금액     30324 non-null  float64
 14  문화_지출_총금액     30324 non-null  float64
 15  교육_지출_총금액     30324 non-null  float64
 16  유흥_지출_총금액     30324 non-null  float64
dtypes: float64(12), int64(2), object(3)
memory usage: 4.0+ MB
